# Hands-on: Build AI Apps with RAG using watsonx.ai, LangChain & Vector Database

## Overview

이 실습에서는 LLM 애플리케이션 구축을 위한 프레임워크인 LangChain을 사용하게 됩니다.
다음 내용을 학습합니다.:
1. Simple Prompt to LLM using LangChain
2. Zero-Shot Prompt and Few-Shot Prompt using Prompt Template
3. Sequential Prompts using Simple Sequential Chain
4. Retrieval Question Answering (RAG)
5. Documents Summarization

## 1. Simple Prompt to LLM (수동으로 프롬프트 템플릿 생성)
- watsonx에서 LLM에 프롬프트를 보내는 기본 사용 사례(Langchain을 사용하지 않음)
- 이 예에서는 LLM 모델(Google flan-ul2)에 직접 간단한 프롬프트를 보냅니다.

### 종속 항목 설치 및 가져오기

In [ ]:
# Install library
!pip install chromadb==0.4.2
!pip install langchain==0.0.312
!pip install langchain --upgrade
!pip install flask-sqlalchemy --user
!pip install pypdf 
!pip install sentence-transformers
!pip install langchain_openai
!pip install -U langchain-ibm

In [ ]:
!pip install -U langchain-community
!pip install -U langchain-ibm

In [ ]:
# Import libraries
import os
import warnings

#from dotenv import load_dotenv
from time import sleep
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM
from langchain import PromptTemplate # Langchain Prompt Template
from langchain.chains import LLMChain, SimpleSequentialChain # Langchain Chains
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator # Vectorize db index with chromadb
from langchain.embeddings import HuggingFaceEmbeddings # For using HuggingFace embedding models
from langchain.text_splitter import CharacterTextSplitter # Text splitter

warnings.filterwarnings("ignore")

### WML 인증정보 설정하기
이 셀은 watsonx Foundation Model 추론 작업에 필요한 WML 자격 증명을 위해 IBM Cloud API 키를 사용합니다.

**Action:** IBM Cloud의 사용자 API 키를 생성합니다. 자세한 내용은 다음을 참조하세요.
[documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui).

In [ ]:
# Get API key and URL from .env
#load_dotenv()
api_key = "<YOUR API KEY HERE>"

### watsonx.ai 플랫폼의 project id 정보 설정하기
Foundation Model 호출을 위해서는 watsonx.ai 플랫폼에서 생성한 프로젝트의 ID가 필요합니다.  
Cloud region

In [ ]:


# region에 따라 주소가 다를 수 있습니다. 주소를 확인해 주세요.
ibm_cloud_url = "https://us-south.ml.cloud.ibm.com" 
project_id = "<YOUR PROJECT ID HERE>"

if api_key is None or ibm_cloud_url is None or project_id is None:
    raise Exception("One or more environment variables are missing!")
else:
    creds = {
        "url": ibm_cloud_url,
        "apikey": api_key 
    }

In [ ]:
# watsonx model 초기화
params = {
    GenParams.DECODING_METHOD: "sample",
    GenParams.TEMPERATURE: 0.2,
    GenParams.TOP_P: 1,
    GenParams.TOP_K: 25,
    GenParams.REPETITION_PENALTY: 1.0,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 20
}

llm_model = Model(
    model_id="google/flan-ul2",
    params=params,
    credentials=creds,
    project_id=project_id
)

print("Done initializing LLM.")

In [ ]:
# simple prompt를 모델에 전송하기
countries = ["France", "Japan", "Australia"]

try:
  for country in countries:
    question = f"What is the capital of {country}"
    res = llm_model.generate_text(question)
    print(f"The capital of {country} is {res.capitalize()}")
except Exception as e:
  print(e)

## 2. Zero-Shot Prompt and Few-Shot Prompt using LangChain's Prompt Template
- 실제 사용 사례는 더 복잡할 수 있습니다. LLM에 일반 프롬프트를 보내는 대신 Langchain 프롬프트 템플릿을 사용하고 있습니다.
- 이 예에서는 Langchain 프롬프트 템플릿을 사용하여 LLM 모델(Google flan-ul2)에 프롬프트를 보냅니다.
- 프롬프트 템플릿 사용의 장점:
    1. **모듈성**: 프롬프트 템플릿을 사용하면 구조화된 템플릿을 한 번 정의하고 이를 다른 입력 변수와 함께 재사용할 수 있습니다. 이렇게 하면 코드가 더 모듈화되고 유지 관리가 더 쉬워집니다.
     2. **동적 입력**: 프롬프트 템플릿에서는 이 예의 "국가"와 같은 동적 입력 변수를 허용합니다. 이는 전체 프롬프트 구조를 수정하지 않고도 입력 값을 쉽게 변경할 수 있음을 의미합니다.
     3. **가독성**: 템플릿은 프롬프트에 명확한 구조를 제공하므로 다른 개발자가 프롬프트의 목적과 프롬프트가 모델과 상호 작용하는 방식을 더 쉽게 이해할 수 있습니다.
     4. **유연성**: 특정 사용 사례나 도메인 요구 사항에 맞게 템플릿을 사용자 정의할 수 있습니다. 이러한 유연성을 통해 전체 프롬프트 논리를 다시 작성하지 않고도 프롬프트를 다양한 시나리오에 적용할 수 있습니다.

### 2.1 Zero-shot Prompt
- Zero-shot Prompt는 가장 간단한 유형의 프롬프트입니다. 모델에 대한 예제는 제공하지 않고 지침만 제공합니다.
- 지시 사항을 질문으로 표현할 수 있습니다. 예: *"제너레이티브 AI의 개념을 설명하세요."*
- 모델에게 '역할'을 부여할 수도 있습니다. 예: *"당신은 데이터 과학자입니다. 생성 AI의 개념을 설명하십시오."*

In [ ]:
# prompt template 정의
prompt = PromptTemplate(
  input_variables = ["country"],
  template = "What is the capital of {country}?",
)

try:
  # Langchain을 사용하려면 Langchain 확장을 인스턴스화해야 합니다.
  lc_llm_model = WatsonxLLM(model=llm_model)
  
  # 생성한 모델과 프롬프트를 기반으로 하는 체인 정의
  chain = LLMChain(llm=lc_llm_model, prompt=prompt)

  # Getting predictions
  countries = ["France", "Japan", "Australia"]
  for country in countries:
    response = chain.run(country)
    print(prompt.format(country=country) + " = " + response.capitalize())
    sleep(0.5)
except Exception as e:
  print(e)

### 2.2 Few-shot Prompt
- Few-shot Prompt는 향후 유사한 작업을 처리하는 방법을 파악하기 위해 모델에 몇 가지 예를 제공합니다.
- 모델이 작업을 더 잘 이해하는 데 도움이 됩니다.

In [ ]:
from langchain.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate

# Few -shot 예제를 입력합니다.
examples = [
    {"input": "What is the capital of Sweden?", "output": "Stockholm"},
    {"input": "What is the capital of Malaysia?", "output": "Kuala Lumpur"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [('human', '{input}'), ('ai', '{output}')]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        #('system', 'You are a helpful AI Assistant'),
        few_shot_prompt,
        ('human', '{input}'),
    ]
)

In [ ]:
try:
  # Langchain을 사용하려면 Langchain 확장을 인스턴스화해야 합니다.
  lc_llm_model = WatsonxLLM(model=llm_model)
  
  # Define a chain based on model and prompt
  chain = LLMChain(llm=lc_llm_model, prompt=final_prompt)

  # LLM에 추론합니다.
  countries = ["France", "Japan", "Australia"]
  for country in countries:
    prompt = f"What is the capital of {country}?"
    print(prompt)
    response = chain.run(prompt)
    print(response)
    #print(prompt.format(country=country) + " = " + response.capitalize())
    sleep(0.5)
except Exception as e:
  print(e)

In [ ]:
from langchain.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate

# Few -shot 예제를 입력합니다.
examples = [
    {"input": "What is the capital of Sweden?", "output": "The capital of Sweden is Stockholm"},
    {"input": "What is the capital of Malaysia?", "output": "The capital of Malaysia is Kuala Lumpur"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [('human', '{input}'), ('ai', '{output}')]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        #('system', 'You are a helpful AI Assistant'),
        few_shot_prompt,
        ('human', '{input}'),
    ]
)

In [ ]:
try:
  # Langchain을 사용하려면 Langchain 확장을 인스턴스화해야 합니다.
  lc_llm_model = WatsonxLLM(model=llm_model)
  
  # 생성한 모델과 프롬프트를 기반으로 하는 체인 정의
  chain = LLMChain(llm=lc_llm_model, prompt=final_prompt)

  # Getting predictions
  countries = ["France", "Japan", "Australia"]
  for country in countries:
    prompt = f"What is the capital of {country}?"
    print(prompt)
    response = chain.run(prompt)
    print(response)
    #print(prompt.format(country=country) + " = " + response.capitalize())
    sleep(0.5)
except Exception as e:
  print(e)

## 3. Sequential Prompts using Simple Sequential Chain
- LangChain의 Simple Sequential Chain을 사용하면 여러 프롬프트를 쉽게 연결하여 순차적 프롬프트를 만들 수 있습니다.
- 순차 프롬프트라고도 하는 프롬프트 체인을 사용하면 하나의 프롬프트에 대한 응답이 시퀀스의 다음 프롬프트에 대한 입력이 될 수 있습니다.
- 각각의 후속 프롬프트는 AI의 이전 응답을 통해 정보를 받아 모델의 출력을 점진적으로 개선하는 일련의 상호 작용을 생성합니다.
- 참조사이트: [SimpleSequentialChain](https://api.python.langchain.com/en/latest/chains/langchain.chains.sequential.SimpleSequentialChain.html)

In [ ]:
# 두개의 sequential prompts 생성

# 주어진 토픽에 대해 랜덤한 질문을 생성하는 프롬프트 템플릿
pt1 = PromptTemplate(input_variables=["topic"], template="Generate a random question about {topic}: Question: ")

# 주어진 질문에 답변하는 프롬프트 템플릿
pt2 = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question: {question}",
)

In [ ]:
# 2개의 모델 인스턴스화(참고, 사용 사례에 따라 모델이 다를 수 있음)
# 위와 같이 WatsonxLLM 래퍼를 반환하는 .to_langchain() 메서드에 주목하세요.
model_1 = Model(
    model_id="google/flan-ul2",
    params=params,
    credentials=creds,
    project_id=project_id
).to_langchain()

model_2 = Model(
    model_id="google/flan-ul2",
    credentials=creds,
    project_id=project_id
).to_langchain()

In [ ]:
# sequential chain 실행
prompt_to_model_1 = LLMChain(llm=model_1, prompt=pt1)
prompt_to_model_2 = LLMChain(llm=model_2, prompt=pt2)
qa = SimpleSequentialChain(chains=[prompt_to_model_1, prompt_to_model_2], verbose=True)

In [ ]:
# "animal"이라는 주제로 체인을 운영합니다.
# 출력을 확인하기 위해 다양한 주제를 제공하면서 놀아보세요. 예. car, the Roman empire
try:
  qa.run("an animal")
except Exception as e:
  print(e)

## 4. Retrieval Question Answering (RAG)
- LangChain의 검색 질문 응답(QA)을 사용하면 문서에서 프롬프트(질문)에 대한 답변으로 쉽게 구절을 추출할 수 있습니다.


### 영어로 된 문서에 질의하기
- 먼저, 이 링크에서 샘플 PDF 파일을 다운로드하세요.: [what_is_generative_ai.pdf](https://ibm.box.com/v/what-is-generative-ai)
- 그런 다음 파일을 프로젝트에 업로드하고 액세스 토큰을 만듭니다.

In [ ]:
# Import library
from ibm_watson_studio_lib import access_project_or_space
from langchain.chains import RetrievalQA

In [ ]:
# Create access token in project
token = "<YOUR ACCESS TOKEN HERE>"
wslib = access_project_or_space({"token":token})
wslib.download_file("what_is_generative_ai.pdf")

In [ ]:
# Load PDF document
pdf = 'what_is_generative_ai.pdf'
loaders = [PyPDFLoader(pdf)]

In [ ]:
# Index loaded PDF
index = VectorstoreIndexCreator(
    embedding = HuggingFaceEmbeddings(),
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)).from_loaders(loaders)

In [ ]:
# watsonx google/flan-ul2 model 초기화
params = {
    GenParams.DECODING_METHOD: "sample",
    GenParams.TEMPERATURE: 0.2,
    GenParams.TOP_P: 1,
    GenParams.TOP_K: 100,
    GenParams.MIN_NEW_TOKENS: 50,
    GenParams.MAX_NEW_TOKENS: 300
}

model = Model(
    model_id="google/flan-ul2",
    params=params,
    credentials=creds,
    project_id=project_id
).to_langchain()

In [ ]:
# RAG chain 초기화
chain = RetrievalQA.from_chain_type(llm=model, 
                                    chain_type="stuff", 
                                    retriever=index.vectorstore.as_retriever(), 
                                    input_key="question")

In [ ]:
# document 기반 질의
res = chain.run("What is Machine Learning?")
print(res)

In [ ]:
# document 기반 질의
res = chain.run("What are the problems generative AI can solve?")
print(res)

In [ ]:
# document 기반 질의
res = chain.run("What are the risks of Generative AI?")
print(res)

### 한글 문서에 질의 하기

In [ ]:
wslib = access_project_or_space({"token":token})
wslib.download_file("ibk_card.pdf")

In [ ]:
# PDF document 로딩
pdf = 'ibk_card.pdf'
loaders = [PyPDFLoader(pdf)]

In [ ]:
model_name = "distiluse-base-multilingual-cased-v1"

index = VectorstoreIndexCreator(
        embedding=HuggingFaceEmbeddings(model_name=model_name),
        text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0)).from_loaders(loaders)

In [ ]:
params = {
    GenParams.DECODING_METHOD: "sample",
    GenParams.TEMPERATURE: 0.2,
    GenParams.TOP_P: 1,
    GenParams.TOP_K: 100,
    GenParams.MIN_NEW_TOKENS: 50,
    GenParams.MAX_NEW_TOKENS: 3000
}

# 한글 지원 모델로 변경
model = Model(
    model_id="meta-llama/llama-2-70b-chat",
    params=params,
    credentials=creds,
    project_id=project_id
).to_langchain()

In [ ]:
# RAG chain 초기화
chain = RetrievalQA.from_chain_type(llm=model, 
                                    chain_type="stuff", 
                                    retriever=index.vectorstore.as_retriever(), 
                                    input_key="question")

In [ ]:
# document 기반 질문하기
res = chain.run("카드사가 부가서비스를 변경할 수 있어?")
print(res)

In [ ]:
# document 기반 질문하기
res = chain.run("카드사가 부가서비스를 변경할 경우 고지방법은?")
print(res)

In [ ]:
# document 기반 질문하기
res = chain.run("일시불 거래 연체 시 어떻게 되?")
print(res)

## 5. Documents Summarization
- 텍스트 요약은 긴 텍스트에 대해 짧지만 유익한 요약을 만드는 NLP의 작업입니다. LLM은 뉴스 기사, 연구 논문, 기술 문서 및 기타 종류의 텍스트를 요약하는 데 사용할 수 있습니다.
- 긴 문서를 요약하는 것은 어려울 수 있습니다. 요약을 생성하려면 색인된 문서에 요약 전략을 적용해야 합니다.
- 이 예에서는 다음 3개 웹사이트의 긴 문서를 요약합니다.
     - https://www.gttkorea.com/news/articleView.html?idxno=9138
     - https://www.hankyung.com/article/202405136945Y
     - https://www.apple-economy.com/news/articleView.html?idxno=73093
- 요약기 앱을 구축할 때 문서를 LLM의 컨텍스트 창으로 전달하는 방법은 다음과 같습니다.
     1. **방법 1: Stuff** - "Stuff"는 전체 문서 내용을 단일 프롬프트에 포함하면 됩니다. (가장 간단한 방법)
     2. **방법 2: MapReduce** - "Map" 단계에서 각 문서를 자체적으로 요약한 다음, "Reduce" 단계에서 최종 요약 합니다.

In [ ]:
# Install library
!pip3 install transformers chromadb langchain

In [ ]:
# Import libraries
import os
from dotenv import load_dotenv
from langchain.document_loaders import WebBaseLoader
from langchain.chains.summarize import load_summarize_chain
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM

### 5.1 Method 1: Stuff
- 단순히 모든 문서를 단일 프롬프트로 “넣는” 방식입니다. 이는 가장 간단한 접근 방식입니다.
- 체인의 `chain_type` 모드를 `stuff`설정 합니다.

### Stuff without using Prompt Template
- 프롬프트 및 LLM 파이프라인은 `load_summarize_chain`이라는 단일 개체로 래핑됩니다.
- `stuff`를 `chain_type`으로 설정합니다.
- 이 예에서는 비교적 짧은 문서가 성공적으로 요약되는 것을 볼 수 있습니다.
- 문서 : [AI 기술 발전에 따라 ‘AI 거버넌스’ 요구도 급증세](https://www.gttkorea.com/news/articleView.html?idxno=9138)

In [ ]:
# document loader 초기화
loader = WebBaseLoader("https://www.gttkorea.com/news/articleView.html?idxno=9138")
doc = loader.load()

# watsonx google/flan-t5-xxl 모델 초기화
# 결과를 최적화하기 위해 일부 런타임 매개변수를 조정해야 할 수도 있습니다.

params = {
    GenParams.DECODING_METHOD: "greedy",
    GenParams.REPETITION_PENALTY: 1.1,
    GenParams.MIN_NEW_TOKENS: 20,
    GenParams.MAX_NEW_TOKENS: 1024
}

llm_model = Model(
    model_id="meta-llama/llama-3-70b-instruct", 
    params=params,
    credentials=creds,
    project_id=project_id
).to_langchain()

# chain_type을 'stuff' 설정
chain = load_summarize_chain(llm_model, chain_type="stuff")

#summarization task 실행
res = chain.run(doc)
print(res)

### Stuff using Prompt Template
- 문서를 프롬프트 템플릿에 로드하고 "stuffed document chain"을 실행합니다. 문서 목록도 채울 수 있습니다.
- `StuffDocumentsChain`은 `load_summarize_chain` 메소드의 일부로 사용됩니다.
- 이 예에서는 위와 동일한 요약 출력이 표시됩니다.
- 참조사이트: [StuffDocumentsChain](https://api.python.langchain.com/en/latest/chains/langchain.chains.combine_documents.stuff.StuffDocumentsChain.html#langchain.chains.combine_documents.stuff.StuffDocumentsChain)

In [ ]:
#Import librararies
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain

# prompt 정의
# <|user|> 는 랭체인 마크업언어로 사용자 입력을 나타내는 마크입니다.
# <|assistant|>는 모델 출력을 나태내는 마크 입니다.
prompt_template = """
<|user|>
다음에 주어진 문서에 대해서 간결한 요약을 작성하시오. 반드시 한국어로 답변하시오.
일본어 또는 중국어 한자어가 포함되어 있으면 한국어로 말하시오.
예) 需求 -> 필요

문서: "{text}"

<|assistant|>
한국어 요약 결과:"""
prompt = PromptTemplate.from_template(prompt_template)

# LLMs chain 정의
llm_chain = LLMChain(llm=llm_model, prompt=prompt)

# StuffDocumentsChain 정의
stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain, document_variable_name="text"
)

# summarization task 실행
res = stuff_chain.run(doc)
print(res)

### LLM 토큰 제한으로 인한 'Stuff' 방법의 제한
- 이 예에서는 더 많은 문서를 추가하면(토큰이 늘어남) 다음 오류가 발생하는 것을 볼 수 있습니다. `입력 토큰 수 8649는 이 모델에 대한 총 토큰 제한 8192를 초과할 수 없습니다.`
- 이는 모델의 토큰 제한(최대 컨텍스트 창 길이) 때문입니다.
- LangChain을 사용하면 청크 및 재귀적 요약 방법을 실행하는 `MapReduce`를 사용하여 이 문제를 해결할 수 있습니다.

In [ ]:
# Load a new document from URL
loader_2 = WebBaseLoader('https://www.hankyung.com/article/202405136945Y')
doc_2 = loader_2.load()

# Combine the new document to the previous document
docs = doc + doc_2

# Run the stuff chain
try:
  res = stuff_chain.run(docs)
  print(res)
except Exception as e:
  print(e)

### 5.2 Method 2: MapReduce
- 이 방법은 각 문서를 `map` 단계에서 개별적으로 요약한 다음, `reduce` 단계에서 요약본들을 최종 요약본으로 합치는 방식입니다.
- Reference: [ReduceDocumentsChain](https://api.python.langchain.com/en/latest/chains/langchain.chains.combine_documents.reduce.ReduceDocumentsChain.html#langchain.chains.combine_documents.reduce.ReduceDocumentsChain)
- Reference: [MapReduceDocumentsChain](https://api.python.langchain.com/en/latest/chains/langchain.chains.combine_documents.map_reduce.MapReduceDocumentsChain.html#langchain.chains.combine_documents.map_reduce.MapReduceDocumentsChain)

In [ ]:
from transformers import AutoTokenizer
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ReduceDocumentsChain, MapReduceDocumentsChain
from time import perf_counter

# Add a 3rd document
print("Loading 3rd document...")
loader_3 = WebBaseLoader("https://www.apple-economy.com/news/articleView.html?idxno=73093")
doc_3 = loader_3.load()
docs = docs + doc_3

# Map
map_template = """
<|user|>
여러개의 문서가 다음과 같이 주어집니다.
{docs}
해당 문서들의 주요 논점을 판단하세요.

<|assistant|>
한국어로 된 요약문을 생성하세요.
한국어 답변 생성: 이 문서는 """
map_prompt = PromptTemplate.from_template(map_template)
print("Init map chain...")
map_chain = LLMChain(llm=llm_model, prompt=map_prompt)

# Reduce
reduce_template = """
<|user|>
다음에 주어지는 내용은 요약된 내용입니다.
{doc_summaries}
해당 내용을 활용하여 최종으로 통합된 주요 주제를 요약하세요.

<|assistant|>
한국어로 된 최종 요약문을 생성하세요.
한국어 답변 생성: 이 문서는 """

reduce_prompt = PromptTemplate.from_template(reduce_template)
print("Init reduce chain...")
reduce_chain = LLMChain(llm=llm_model, prompt=reduce_prompt)

# 문서 목록을 가져와 단일 문자열로 결합한 다음 이를 LLMChain에 전달합니다.
print("Stuff documents using reduce chain...")
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="doc_summaries"
)

# 매핑된 문서를 결합하고 반복적으로 축소합니다.
reduce_documents_chain = ReduceDocumentsChain(
    # 이것이 호출되는 최종 체인입니다.
    combine_documents_chain=combine_documents_chain,
    # 문서가 'StuffDocumentsChain'의 컨텍스트를 초과하는 경우
    collapse_documents_chain=combine_documents_chain,
    # 문서를 그룹화할 최대 토큰 수입니다.
    token_max=4000
)

# 체인을 매핑하여 문서를 결합한 다음 결과를 결합합니다.
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # 문서를 넣을 llm_chain의 변수 이름
    document_variable_name="docs",
    # 출력에 맵 단계의 결과를 반환합니다.
    return_intermediate_steps=True,
    verbose=False
)

# 여기서는 특히 flan-ul2 모델을 위해 Huggingface의 사전 훈련된 토크나이저를 사용하고 있습니다.
# 결과가 어떻게 변하는지 보기 위해 다양한 토크나이저와 텍스트 분할기를 사용해 볼 수도 있습니다.
print("Init chunk splitter...")
try:
    tokenizer = AutoTokenizer.from_pretrained("intfloat/multilingual-e5-base") # Hugging face tokenizer for flan-ul2
    text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
        tokenizer=tokenizer
    )
    split_docs = text_splitter.split_documents(docs)
    print(f"Using {len(split_docs)} chunks: ")
except Exception as ex:
    print(ex)

print("Run map-reduce chain. This should take ~15-30 seconds...")
try:
    t1_start = perf_counter()
    results = map_reduce_chain(split_docs)
    steps = results["intermediate_steps"]
    output = results["output_text"]
    t1_stop = perf_counter()
    print("Elapsed time:", round((t1_stop - t1_start), 2), "seconds.\n") 

    print("Results from each chunk: \n")
    for idx, step in enumerate(steps):
        print(f"{idx + 1}. {step}\n")
    
    print("\n\nFinal output:\n")
    print(output)

    print("\nDone.")
except Exception as e:
    print(e)

- 보시다시피 `Langchain`은 모델용 토크나이저와 함께 더 많은 양의 텍스트를 신속하게 덩어리로 나누고 간결한 문장 한두 개로 `반복적`으로 요약할 수 있습니다. 다양한 문서를 시도하고, 모델 런타임 매개변수를 조정하고, 다른 모델을 모두 시도하여 상황이 어떻게 작동하는지 확인해 볼 수 있습니다. 좋은 결과를 얻기 위해 주목해야 할 가장 중요한 사항 중 하나는 입력이 `청크화`되고 `토큰화`되는 방식이 매우 중요하다는 것입니다. 좋지 않은 `map` 결과를 전달하면 요약 품질이 낮아집니다.